In [4]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

# Import data
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)
# Define loss and optimizer
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

with tf.name_scope('reshape'):
  x_image = tf.reshape(x, [-1, 28, 28, 1])

# First convolutional layer - maps one grayscale image to 32 feature maps.
with tf.name_scope('conv1'):
  h_conv1 = tf.layers.conv2d(x_image, 32, [5,5],
                             padding='SAME',
                             activation=tf.nn.relu)
 

# Pooling layer - downsamples by 2X.
with tf.name_scope('pool1'):
  h_pool1 = tf.layers.max_pooling2d(h_conv1, pool_size=[2,2],
                        strides=[2, 2], padding='VALID')

# Second convolutional layer -- maps 32 feature maps to 64.
with tf.name_scope('conv2'):
  h_conv2 = tf.layers.conv2d(h_pool1, 64, [5,5],
                             padding='SAME',
                             activation=tf.nn.relu)

# Second pooling layer.
with tf.name_scope('pool2'):
  h_pool2 = tf.layers.max_pooling2d(h_conv2, pool_size=[2,2],
                        strides=[2, 2], padding='VALID')

# Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
# is down to 7x7x64 feature maps -- maps this to 1024 features.
with tf.name_scope('fc1'):
  h_pool2_flat = tf.layers.flatten(h_pool2)
  h_fc1 = tf.layers.dense(h_pool2_flat, 1024, activation=tf.nn.relu)

# Dropout - controls the complexity of the model, prevents co-adaptation of
# features.
with tf.name_scope('dropout'):
  keep_prob = tf.placeholder(tf.float32)
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Map the 1024 features to 10 classes, one for each digit
with tf.name_scope('fc2'):
  y = tf.layers.dense(h_fc1_drop, 10, activation=None)





# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
# Train
for step in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  lr = 0.01
  _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
  if (step+1) % 100 == 0:
    print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
  if (step+1) % 1000 == 0:
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
step 100, entropy loss: 1.902616, l2_loss: 805.193054, total loss: 1.958979
0.56
step 200, entropy loss: 0.664573, l2_loss: 806.976257, total loss: 0.721062
0.78
step 300, entropy loss: 0.634558, l2_loss: 807.991516, total loss: 0.691118
0.85
step 400, entropy loss: 0.368051, l2_loss: 808.563782, total loss: 0.424651
0.93
step 500, entropy loss: 0.349145, l2_loss: 809.038940, total loss: 0.405778
0.88
step 600, entropy loss: 0.281290, l2_loss: 809.375916, total loss: 0.337947
0.93
step 700, entropy loss: 0.242394, l2_loss: 809.697266, total loss: 0.299072
0.95
step 800, entropy loss: 0.168864, l2_loss: 809.989990, total loss: 0.225563
0.99
step 900, entropy loss: 0.256524, l2_loss: 810.257202, total loss: 0.3